In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
#main function to run all others, takes a *l_border* and a *r_border*  in which it operates
def main():
    global axis
    models = open('models.txt', 'r')
    Lines = models.readlines()
    line_number = 1
    for line in Lines:
        if line_number == 1:
            file = line.split()
            models_directory = file[0]
            line_number += 1
        else:
            file = line.split()
            #imports both text files with coordinates, make sure the files are formated correctly before importing
            bone1 = pd.read_csv(models_directory+file[0], names=('X', 'Y', 'Z', 'Indicator'), delim_whitespace=True, dtype={'X': np.float64, 'Y': np.float64, 'Z': np.float64, 'Indicator': np.float64})
            bone2 = pd.read_csv(models_directory+file[1], names=('X', 'Y', 'Z', 'Indicator'), delim_whitespace=True, dtype={'X': np.float64, 'Y': np.float64, 'Z': np.float64, 'Indicator': np.float64})

            length1 = len(bone1.index)
            length2 = len(bone2.index)

            #a 1 in the indicator column indicates a coordinate belongs to the first bone while a 0 indicates it belongs to the second bone
            for i in range(length1):
                bone1['Indicator'][i] = 1

            for i in range(length2):
                bone2['Indicator'][i] = 0

            bonesx = pd.concat([bone1, bone2])
            bonesy = pd.concat([bone1, bone2])
            bonesz = pd.concat([bone1, bone2])
            lengthu =  len(bonesx.index)
            l_border = 0
            r_border = lengthu-1

            bonesx.sort_values(by=['X'], inplace=True)
            bonesy.sort_values(by=['Y'], inplace=True)
            bonesz.sort_values(by=['Z'], inplace=True)

            unionx = bonesx.to_numpy()
            uniony = bonesy.to_numpy()
            unionz = bonesz.to_numpy()
            
            intersection_points = set()        
            axis_specific_julius(0, lengthu-1, intersection_points, 'x', unionx, uniony, unionz)
            lengthx = len(intersection_points)
            intersection_points = set()        
            axis_specific_julius(0, lengthu-1, intersection_points, 'y', unionx, uniony, unionz)
            lengthy = len(intersection_points)
            intersection_points = set()        
            axis_specific_julius(0, lengthu-1, intersection_points, 'z', unionx, uniony, unionz)
            lengthz = len(intersection_points)
            print('line number is:%s x:%s y:%s z:%s' % (line_number, lengthx, lengthy, lengthz))

            if lengthx <= lengthy and lengthx <= lengthz:
                print('x axis was the shortes axis')
                union = unionx
                axis = 'x'
                distance = float(file[2])

            elif lengthy <= lengthx and lengthy <= lengthz:
                print('y axis was the shortes axis')
                union = uniony
                axis = 'y'
                distance = float(file[3])
                
            elif lengthz <= lengthy and lengthz <= lengthx:
                print('z axis was the shortes axis')
                union = unionz
                axis = 'z'
                distance = float(file[4])
            
            filename = file[0] + '_and_' + file[1] + 'at' + str(distance) + 'dist.txt'
                
            intersection_points = set()
            julius(l_border, r_border, intersection_points, union)
            print('finished julius at %s, found %s intersecting points' % (datetime.now()-startTime, len(intersection_points)))
            intersection_points_list = list(intersection_points)
            intersection_points_list.sort()
            poi = np.array(intersection_points_list)
            pathfinder(poi, filename, distance, union, axis)
            lines_seen = set() # holds lines already seen
            outfile = open('no_dups_' + filename, "w")
            for suture_line in open(filename, "r"): #loop to remove duplicate lines
                if suture_line not in lines_seen: # not a duplicate
                    outfile.write(suture_line)
                    lines_seen.add(suture_line)
            outfile.close()
            line_number += 1

In [7]:
#check for a same type of bone in a given range of points, returns True if so. Checks for a specific axis
def axis_specific_same_type_of_bone_check(l_border, r_border, axis, unionx, uniony, unionz):
    addition = 0
    if axis == 'x':
        axis_union = unionx
    elif axis == 'y':
        axis_union = uniony
    elif axis == 'z':
        axis_union = unionz
    for i in range(r_border-l_border+1):
        addition = addition + axis_union[l_border+i, 3]
    try:
        if addition / (r_border-l_border+1) == 1 or addition / (r_border-l_border+1) == 0:
            return True
        else:
            return False
    except ZeroDivisionError:
        print('Zero Division Error was found with l_border %s and r_border %s' % (l_border,r_border))

#recursive function to search for points that border regions with the same type of bone, returns a set of those points *intersection_points*. Checks for a specific axis
def axis_specific_julius(l_border, r_border, intersection_points, axis, unionx, uniony, unionz):
    #print (intersection_points)
    midpoint = l_border + (r_border-l_border)//2
    if axis_specific_same_type_of_bone_check(l_border,r_border, axis, unionx, uniony, unionz):
        #print('only one type of bone, l_border is %s, r_border is %s, checking l_border %s and r_border %s' % (l_border, r_border, l_border, r_border))
        intersection_points.update([l_border, r_border])
        return
    elif axis_specific_same_type_of_bone_check(l_border, midpoint, axis, unionx, uniony, unionz):
        #print('moving right, l_border is %s, r_border is %s; checking l_border %s and midpoint %s' % (l_border, r_border, l_border, midpoint))
        intersection_points.update([midpoint, l_border])
        l_border = midpoint + 1
        axis_specific_julius(l_border, r_border, intersection_points, axis, unionx, uniony, unionz)
    elif axis_specific_same_type_of_bone_check(midpoint + 1, r_border, axis, unionx, uniony, unionz):
        #print('moving left, l_border is %s, r_border is %s; checking midpoint %s and r_border %s' % (l_border, r_border, midpoint+1, r_border))
        intersection_points.update([midpoint + 1, r_border])
        r_border = midpoint -1
        axis_specific_julius(l_border, r_border, intersection_points, axis, unionx, uniony, unionz)
    else:
        #print('moving middle, l_border is %s, r_border is %s' % (l_border, r_border))
        axis_specific_julius(l_border, midpoint, intersection_points, axis, unionx, uniony, unionz)
        axis_specific_julius(midpoint+1, r_border, intersection_points, axis, unionx, uniony, unionz)


In [8]:
#check for a same type of bone in a given range of points, returns True if so
def same_type_of_bone_check(l_border, r_border, union):
    addition = 0
    for i in range(r_border-l_border+1):
        addition = addition + union[l_border+i, 3]
    try:
        if addition / (r_border-l_border+1) == 1 or addition / (r_border-l_border+1) == 0:
            return True
        else:
            return False
    except ZeroDivisionError:
        print('Zero Division Error was found with l_border %s and r_border %s' % (l_border,r_border))

#total number of comparisons for a set amount of points given as *value*, returns the number of comparisons
def number_of_comparisons_in(value):
    number_of_comparisons_in = (value * (value +1)) / 2 
    return number_of_comparisons_in
        
#checks the distance between all points in a given range and outputs to a txt file    
def distance_check(l_border, r_border, output_file, distance, union, axis):
    do_percent = 0 #activates a percentage indicator when doing distance checking
    midpoint = l_border + (r_border-l_border)//2
    if number_of_comparisons_in(midpoint-l_border+1)+number_of_comparisons_in(r_border-midpoint+2)+number_of_comparisons_in(caesar(midpoint, 'length', union, distance, axis))<number_of_comparisons_in(r_border-l_border+1):
        distance_check(l_border,midpoint, output_file, distance, union, axis)
        distance_check(midpoint+1,r_border, output_file, distance, union, axis)
        distance_check(caesar(midpoint, 'l_border', union, distance, axis), caesar(midpoint, 'r_border', union, distance, axis), output_file, distance, union, axis)
    else:
        count = 0
        scount = 0
        percentile = 1
        percent = number_of_comparisons_in(r_border-l_border+1) // 10
        for i in range(r_border-l_border+1):
            for e in range(i+1, r_border-l_border+1):
                count += 1
                if count > percent and do_percent == 1:
                    print('%s percent' % (percentile))
                    percentile += 1
                    count = 0
                if union[l_border + i, 3] != union[l_border + e, 3]:
                    magnitude = np.linalg.norm(union[l_border + i, 0:3] - union[l_border + e, 0:3])
                    if magnitude < distance:
                        scount += 1
                        #print('a suture was found at positions %s and %s: %s and %s' % (l_border + i, l_border + e, union[l_border + i, 0:4], union[l_border + e, 0:4]))
                        output_file.write('%s %s %s\n' % (union[l_border + i, 0], union[l_border + i, 1], union[l_border + i, 2]))
                        output_file.write('%s %s %s\n' % (union[l_border + e, 0], union[l_border + e, 1], union[l_border + e, 2]))
        print('Found %s sutures out of %s comparisons from points %s to %s at %s' % (scount, number_of_comparisons_in(r_border-l_border+1), l_border, r_border, datetime.now()-startTime))

#for a given point *border* it obtains the leftmost point and the rightmost point within the set Z axis distance, three available flags
def caesar(border, flag, union, distance, axis):
    lengthu = len(union)
    searchn = 1
    searchp = 1
    if axis == 'x':
        xyz = 0
    elif axis == 'y':
        xyz = 1
    elif axis == 'z':
        xyz = 2
    perimetern = union[border, xyz] - distance
    perimeterp = union[border, xyz] + distance
    begin_indexn = 0
    r_border_indexn = lengthu
    #print('borderz is %s, perimetern is %s, perimeterp is %s' % (borderz, perimetern, perimeterp))
    while searchn == 1:
        midpointn = begin_indexn + (r_border_indexn-begin_indexn)//2
        midpoint_valuen = union[midpointn, xyz]
        premidpoint_valuen = union[midpointn-1, xyz]
        #print('checking midpoint %s, midpoint value is %s, premidpoint value is %s' % (midpointn, midpoint_valuen, premidpoint_valuen))
        if midpointn == 0:
            l_border = midpointn
            searchn = 0
            r_borderpos = l_border
            #print('found l_border')
            while searchp == 1:
                if r_borderpos == lengthu-1:
                    r_border = r_borderpos
                    searchp = 0
                elif union[r_borderpos, xyz] > perimeterp:
                    r_border = r_borderpos-1
                    searchp = 0
                r_borderpos += 1
                
        elif midpoint_valuen >= perimetern and premidpoint_valuen < perimetern:
            l_border = midpointn
            searchn = 0
            r_borderpos = l_border
            #print('found l_border')
            while searchp == 1:
                if r_borderpos == lengthu-1:
                    r_border = r_borderpos
                    searchp = 0
                elif union[r_borderpos, xyz] > perimeterp:
                    r_border = r_borderpos-1
                    searchp = 0
                r_borderpos += 1
                
        elif perimetern < midpoint_valuen or premidpoint_valuen == perimetern:
            r_border_indexn = midpointn-1
            
        elif perimetern > midpoint_valuen:
            begin_indexn = midpointn+1
            
    #print('l_border is %s, r_border is %s' % (l_border, r_border))
    if flag == 'length': #returns the amount of points within the range
        return r_border-l_border+1
    elif flag == 'l_border':
        return l_border #returns the leftmost border
    elif flag == 'r_border':
        return r_border #returns the rightmost border
    
#recursive function to search for points that border regions with the same type of bone, returns a set of those points *intersection_points*
def julius(l_border, r_border, intersection_points, union):
    #print (intersection_points)
    midpoint = l_border + (r_border-l_border)//2
    if same_type_of_bone_check(l_border,r_border, union):
        #print('only one type of bone, l_border is %s, r_border is %s, checking l_border %s and r_border %s' % (l_border, r_border, l_border, r_border))
        intersection_points.update([l_border, r_border])
        return
    elif same_type_of_bone_check(l_border, midpoint, union):
        #print('moving right, l_border is %s, r_border is %s; checking l_border %s and midpoint %s' % (l_border, r_border, l_border, midpoint))
        intersection_points.update([midpoint, l_border])
        l_border = midpoint + 1
        julius(l_border, r_border, intersection_points, union)
    elif same_type_of_bone_check(midpoint + 1, r_border, union):
        #print('moving left, l_border is %s, r_border is %s; checking midpoint %s and r_border %s' % (l_border, r_border, midpoint+1, r_border))
        intersection_points.update([midpoint + 1, r_border])
        r_border = midpoint -1
        julius(l_border, r_border, intersection_points, union)
    else:
        #print('moving middle, l_border is %s, r_border is %s' % (l_border, r_border))
        julius(l_border, midpoint, intersection_points, union)
        julius(midpoint+1, r_border, intersection_points, union)


#takes the set of intersecting points and finds the optimum way with least amounts of comparisons to check all distances
def pathfinder(poi, filename, distance, union, axis):
    l_border = 0
    r_border = 0
    intervaled_comparisons = 0
    output_file=open(filename, 'a+')
    for i in range(np.size(poi)):
        continous_comparisons = number_of_comparisons_in(caesar(poi[r_border], 'r_border', union, distance, axis)-caesar(poi[l_border], 'l_border', union, distance, axis) +1)
        intervaled_comparisons += number_of_comparisons_in(caesar(poi[i], 'length', union, distance, axis))
        if continous_comparisons <= intervaled_comparisons:
            #print('continous_comparisons %s is smaller then intervaled_comparisons %s. l_border is %s r_border is %s iteration is %s' % (continous_comparisons, intervaled_comparisons, l_border, r_border, i))
            if i == np.size(poi)-1:
                print('continous_comparisons %s continued to be smaller then intervaled_comparisons %s so checking from %s to %s' % (continous_comparisons, intervaled_comparisons, caesar(poi[l_border], 'l_border', union, distance, axis), caesar(poi[r_border], 'r_border', union, distance, axis)))
                distance_check(caesar(poi[l_border], 'l_border', union, distance, axis),caesar(poi[r_border], 'r_border', union, distance, axis), output_file, distance, union, axis)
            else:
                r_border += 1
        else:
            #print( 'continous_comparisons %s is greater then intervaled_comparisons %s. l_border is %s r_border is %s iteration is %s' % (continous_comparisons, intervaled_comparisons, l_border, r_border, i))
            print('checking %s to %s' % (caesar(poi[l_border], 'l_border', union, distance, axis), caesar(poi[r_border-1], 'r_border', union, distance, axis)))
            distance_check(caesar(poi[l_border], 'l_border', union, distance, axis), caesar(poi[r_border-1], 'r_border', union, distance, axis), output_file, distance, union, axis)
            l_border = r_border
            r_border += 1
            intervaled_comparisons = 0
    output_file.close()


In [9]:
#setting startTime when running main insures accurate timetables
startTime = datetime.now()
main()

line number is:2 x:53005 y:66798 z:50400
z axis was the shortes axis
finished julius at 0:00:14.722648, found 50400 intersecting points
checking 0 to 214
Found 0 sutures out of 23220.0 comparisons from points 0 to 214 at 0:00:14.741576
checking 37982 to 40763
Found 0 sutures out of 3871153.0 comparisons from points 37982 to 40763 at 0:00:16.841980
continous_comparisons 6920349481.0 continued to be smaller then intervaled_comparisons 1688264190224.0 so checking from 40576 to 158221
Found 0 sutures out of 6762003.0 comparisons from points 40576 to 44252 at 0:04:56.422299
Found 13 sutures out of 6758326.0 comparisons from points 44253 to 47928 at 0:05:18.569045
Found 0 sutures out of 7374720.0 comparisons from points 42970 to 46809 at 0:05:41.934589
Found 11894 sutures out of 27036981.0 comparisons from points 47929 to 55281 at 0:07:05.322583
Found 4319 sutures out of 17449278.0 comparisons from points 45214 to 51120 at 0:08:02.352081
Found 6586 sutures out of 27036981.0 comparisons from 

KeyboardInterrupt: 